In [1]:
import sys

sys.path.extend(['DeepGMM'])

from DeepGMM.methods.toy_model_selection_method import ToyModelSelectionMethod
from kernel import CategoryKernel, RBFKernel
from model import train_HSIC_IV, NonlinearModel, train_mse
import pandas as pd
from utils import med_sigma, to_torch, gen_data, fit_restart, gen_radial_fn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import functools

In [2]:
np.random.seed(1)

n_rep = 8
n = 1000
num_basis = 10
data_limits = (-7, 7)

config_hsic = {'batch_size': 256, 'lr': 5e-2, 
               'max_epoch': 700, 'num_restart': 1}

config_mse = {'batch_size': 256, 'lr': 5e-2, 
              'max_epoch': 300}

f = gen_radial_fn(num_basis=num_basis, data_limits=data_limits)

In [3]:
# for instrument in ['Gaussian', 'Binary']:
for instrument in ['Gaussian', 'Binary']:
    res_df = None
    ret_df_vis = None
    # get a fix x_vis
    iv_type = 'mix_{}'.format(instrument)
    _, _, _, X_vis = gen_data(f, n, iv_type)

    alphas = np.linspace(0, 1, 5)

    for j in range(len(alphas)):
        alpha = alphas[j]

        def rep_function(i):
            X, Y, Z, _ = gen_data(f, n, iv_type, alpha=alpha)
            # dev set for DeepGMM
            X_dev, Y_dev, Z_dev, _ = gen_data(f, n, iv_type, alpha=alpha)
            # oracle set
            X_o, Y_o, Z_o, _ = gen_data(f, n, iv_type, alpha=alpha, oracle=True)
            X_test, _, _, _ = gen_data(f, X_vis.shape[0], iv_type, alpha=alpha)

            # Pure predictive
            mse_net = NonlinearModel(input_dim=1, 
                                      lr=config_mse['lr'],
                                      lmd=-99)

            mse_net = train_mse(mse_net, config_mse, X, Y, Z)
            y_hat_mse = mse_net(to_torch(X_test)).detach().numpy()

            oracle_net = train_mse(mse_net, config_mse, X_o, Y_o, Z_o)
            y_hat_oracle = mse_net(to_torch(X_test)).detach().numpy()

            # HSIC IV
            s_z = med_sigma(Z)
            kernel_e = RBFKernel(sigma=1)

            if instrument == 'Binary':
                kernel_z = CategoryKernel()
            else:
                kernel_z = RBFKernel(sigma=s_z)
            
            # non regularized HSIC IV
            hsic_net = NonlinearModel(input_dim=1, 
                                      lr=config_hsic['lr'],
                                      kernel_e=kernel_e,
                                      kernel_z=kernel_z,
                                      lmd=0)

            hsic_net.load_state_dict(mse_net)
            hsic_net = train_HSIC_IV(hsic_net, config_hsic, X, Y, Z, verbose=False)

            intercept_adjust = Y.mean() - hsic_net(to_torch(X)).mean()
            y_hat_hsic = intercept_adjust + hsic_net(to_torch(X_test))
            y_hat_hsic = y_hat_hsic.detach().numpy().copy()
            
            # regularized HSIC IV
            hsic_net = NonlinearModel(input_dim=1, 
                                      lr=config_hsic['lr'],
                                      kernel_e=kernel_e,
                                      kernel_z=kernel_z,
                                      lmd=5e-5)

            hsic_net.load_state_dict(mse_net)
            hsic_net = train_HSIC_IV(hsic_net, config_hsic, X, Y, Z, verbose=False)

            intercept_adjust = Y.mean() - hsic_net(to_torch(X)).mean()
            y_hat_hsic_pen = intercept_adjust + hsic_net(to_torch(X_test))
            y_hat_hsic_pen = y_hat_hsic_pen.detach().numpy().copy()

            # prepare data for DeepGMM
            dat = [X, Z, Y, X_dev, Z_dev, Y_dev]
            # to torch
            for k in range(len(dat)):
                dat[k] = to_torch(dat[k]).double()

            deepGMM = ToyModelSelectionMethod()
            deepGMM.fit(*dat, g_dev=None, verbose=True)
            y_hat_deepGMM = deepGMM.predict(to_torch(X_test).double()).flatten().detach().numpy()

            inner_df = pd.DataFrame()
            inner_df_vis = pd.DataFrame()

            inner_df['f_x'] = f(X_test)
            inner_df['Pred'] = y_hat_mse
            inner_df['HSIC-IV'] = y_hat_hsic
            inner_df['HSIC-AR'] = y_hat_hsic_pen
            inner_df['D-GMM'] = y_hat_deepGMM
            inner_df['Oracle'] = y_hat_oracle
            inner_df['alpha'] = alpha
            inner_df['run_id'] = i

            return inner_df
        
        ret_df = Parallel(n_jobs=4)(
            delayed(rep_function)(i=i) for i in range(n_rep))

        # ret_df = [rep_function(i) for i in range(n_rep)]

        ret_df = functools.reduce(lambda df1, df2: df1.append(df2, ignore_index=True), ret_df)

        if res_df is None:
            res_df = ret_df
        else:
            res_df = res_df.append(ret_df, ignore_index=True)

    melt_res_df = res_df.melt(id_vars=['f_x', 'alpha', 'run_id'], var_name='Method',
                              value_name='y_pred')
    melt_res_df['MISE'] = (melt_res_df['f_x'] - melt_res_df['y_pred']) ** 2
    final_df = melt_res_df.groupby(['Method', 'alpha', 'run_id'])['MISE'].mean().reset_index()
    final_df['alpha'] = np.round(final_df.alpha, 2)
    final_df.to_csv("results/compare_df_NN_ins_{}.csv".format(instrument),
                    index=False)
    sns.set(font_scale=1.4, style='white', palette=sns.set_palette("tab10"))

    g = sns.catplot(data=final_df, kind="point", log=True,
                    x='alpha', y='MISE', hue='Method',
                    markers=["o", "x", "d", "s", "v"], linestyles=[':', '--', '-', '-.', ':'],
                    capsize=.07, aspect=1.5, height=3.2, ci=95)
    g.fig.get_axes()[0].set_yscale('log')
    g._legend.remove()

    plt.xlabel(r'$\alpha$')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), 
               ncol=5, fancybox=True, shadow=False, prop={'size': 10})
    plt.savefig('results/compare_NN_ins_{}.pdf'.format(instrument),
                bbox_inches="tight")
    plt.close()


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:429: UserWarning: The number of training s

p-value: 0.9491878725332269
iteration 1, accept the test!
p-value: 0.7973788055822338
iteration 1, accept the test!
starting learning args eval 0
starting learning args eval 1
starting learning args eval 2
learning eval: -0.0008624847958145626 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0025:'betas'=(0.5, 0.9)
learning eval: -0.0003339656069959308 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0002:'betas'=(0.5, 0.9) OAdam:::'lr'=0.001:'betas'=(0.5, 0.9)
learning eval: -0.0003749015649348753 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0001:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9)
size of f_z collection: 300
iteration 0, dev-MSE=0.000000, train-loss=-0.584403, dev-loss=-0.661850, mean-recent-eval=-0.221306
iteration 100, dev-MSE=0.000000, train-loss=0.271197, dev-loss=0.277111, mean-recent-eval=-0.204337
iteration 200, dev-MSE=0.000000, train-loss=0.086104, dev-loss=0.095005, mean-recent-eval=-0.093789
iteration 3

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the

p-value: 0.8987982886495935
iteration 1, accept the test!
p-value: 0.8856752610415047
iteration 1, accept the test!
starting learning args eval 0
starting learning args eval 1
starting learning args eval 2
learning eval: -0.000152941788921957 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0025:'betas'=(0.5, 0.9)
learning eval: -0.000186436692880201 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0002:'betas'=(0.5, 0.9) OAdam:::'lr'=0.001:'betas'=(0.5, 0.9)
learning eval: -0.00022527235791368195 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0001:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9)
size of f_z collection: 300
iteration 0, dev-MSE=0.000000, train-loss=1.096508, dev-loss=0.972433, mean-recent-eval=-0.042228
iteration 100, dev-MSE=0.000000, train-loss=0.000174, dev-loss=0.002517, mean-recent-eval=-0.005997
iteration 200, dev-MSE=0.000000, train-loss=0.001226, dev-loss=0.003522, mean-recent-eval=-0.000402
iteration 300,

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
DeepGMM/optimizers/oadam.py:94: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272178570/work/torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataload

...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
G

p-value: 0.9856915213749604
iteration 1, accept the test!
p-value: 0.9442955582900491
iteration 1, accept the test!
starting learning args eval 0
starting learning args eval 1
starting learning args eval 2
learning eval: 0.015138919867112952 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0025:'betas'=(0.5, 0.9)
learning eval: 0.026146167662252598 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0002:'betas'=(0.5, 0.9) OAdam:::'lr'=0.001:'betas'=(0.5, 0.9)
learning eval: 0.026741148466115597 OptimalObjective::lambda_1=0.250000 OAdam:::'lr'=0.0001:'betas'=(0.5, 0.9) OAdam:::'lr'=0.0005:'betas'=(0.5, 0.9)
size of f_z collection: 300
iteration 0, dev-MSE=0.000000, train-loss=-1.156920, dev-loss=-1.423919, mean-recent-eval=-0.162247
iteration 100, dev-MSE=0.000000, train-loss=-0.168074, dev-loss=-0.233347, mean-recent-eval=-0.191579
iteration 200, dev-MSE=0.000000, train-loss=0.307966, dev-loss=0.358036, mean-recent-eval=-0.206005
iteration 300, 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the


iteration 1800, dev-MSE=0.000000, train-loss=0.000018, dev-loss=0.000504, mean-recent-eval=0.151420
iteration 1900, dev-MSE=0.000000, train-loss=0.000058, dev-loss=0.001081, mean-recent-eval=0.152581
iteration 2000, dev-MSE=0.000000, train-loss=0.000014, dev-loss=0.001026, mean-recent-eval=0.152678
iteration 2100, dev-MSE=0.000000, train-loss=0.000184, dev-loss=-0.001179, mean-recent-eval=0.152623
iteration 2200, dev-MSE=0.000000, train-loss=0.000000, dev-loss=0.000501, mean-recent-eval=0.152517
iteration 2300, dev-MSE=0.000000, train-loss=-0.000007, dev-loss=0.000180, mean-recent-eval=0.152564
iteration 2400, dev-MSE=0.000000, train-loss=0.000009, dev-loss=0.000629, mean-recent-eval=0.152620
iteration 2500, dev-MSE=0.000000, train-loss=0.000008, dev-loss=0.000409, mean-recent-eval=0.152622
iteration 2600, dev-MSE=0.000000, train-loss=-0.000000, dev-loss=-0.001113, mean-recent-eval=0.152518
iteration 2700, dev-MSE=0.000000, train-loss=0.000005, dev-loss=0.000135, mean-recent-eval=0.15

/opt/conda/lib/python3.7/site-packages/seaborn/categorical.py:1784: UserWarning: You passed a edgecolor/edgecolors ((1.0, 0.4980392156862745, 0.054901960784313725)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  zorder=z)


In [9]:
all_df = pd.DataFrame()
for instrument in ['Binary', 'Gaussian']:
# for instrument in ['Gaussian']:
    final_df = pd.read_csv("results/compare_df_NN_ins_{}.csv".format(instrument))
    final_df = final_df.replace({"HSIC-AR": "HSIC-X-pen", 
                                 "HSIC-IV": "HSIC-X",
                                 "Pred":"OLS",
                                 "D-GMM": "DeepGMM"})
    final_df[r'$Z$'] = instrument
    all_df = all_df.append(final_df, ignore_index=True)
    
sns.set(font_scale=1.8, style='white', palette=sns.set_palette("tab10"))

g = sns.catplot(data=all_df, kind="point", log=True,
                x='alpha', y='MISE', hue='Method',
                hue_order=['DeepGMM', 'OLS', 'Oracle', 'HSIC-X', 'HSIC-X-pen'],
                markers=["o", "x", "d", "s", "v"], linestyles=[':', '-', '--', '-.', ':'],
                capsize=.07, aspect=1.2, height=4, ci=95,
                col=r'$Z$', sharey=False)

g._legend.remove()

plt.legend(loc='upper center', bbox_to_anchor=(-.25, 1.43), 
  ncol=5, fancybox=True, shadow=True, prop={'size': 15.5})

for ax in g.axes.flat:
    ax.set_yscale('log')

g.set_xlabels(r'$\alpha$')
g.set_ylabels('MSE')

plt.savefig('results/compare_NN.pdf',
            bbox_inches="tight")
plt.close()

/opt/conda/lib/python3.7/site-packages/seaborn/categorical.py:1784: UserWarning: You passed a edgecolor/edgecolors ((1.0, 0.4980392156862745, 0.054901960784313725)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  zorder=z)
/opt/conda/lib/python3.7/site-packages/seaborn/categorical.py:1784: UserWarning: You passed a edgecolor/edgecolors ((1.0, 0.4980392156862745, 0.054901960784313725)) for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  zorder=z)
